In [1]:
COMPUTE_CV = True

In [2]:
import pandas as pd

# DATA_PATH = "../input/shopee-product-matching/"
DATA_PATH = "../input/"


if COMPUTE_CV == False:
    train = pd.read_csv(DATA_PATH + "test.csv")
    train["image"] = DATA_PATH + "test_images/" + train["image"]
else:
    train = pd.read_csv(DATA_PATH + "train_min.csv")
    train["target"] = train.label_group.map(
        train.groupby("label_group").posting_id.agg("unique").to_dict()
    )
    train["target"] = train["target"].apply(lambda x: " ".join(x))
    train["image"] = DATA_PATH + "train_images/" + train["image"]

train.head()

,posting_id,image,image_phash,title,label_group,target
2,train_2288590299,../input/train_images/000a190fdd715a2a36faed16...,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,train_2288590299 train_3803689425
10,train_1598329973,../input/train_images/001d7f5d9a2fac714f4d5f37...,bec8d09693634b4b,Atasan Rajut Wanita LISDIA SWEATER,2462407944,train_1598329973 train_841015183 train_4224502769
14,train_4287573913,../input/train_images/001f5580b058c6b8e3313219...,dc85e1750687f932,Charger VIZZ VZ-TC11 / batok charger vizz 1A O...,1932232224,train_4287573913 train_1748125201 train_163362...
15,train_4196427721,../input/train_images/002039aaf8618627a0442d5e...,e98c873acc65946e,Korek Kuping LED untuk balita CherryBabyKidsSh...,349297863,train_4196427721 train_1482447822 train_234660...
18,train_2985955659,../input/train_images/002f978c58a44a00aadfca71...,bf38f0e083d7c710,HnKfashion Sweater Hoodie WHO Printing BabyTer...,3415582503,train_2985955659 train_3916258742 train_415673...


In [3]:
import xlm_v1_arcface

train = xlm_v1_arcface.eval(
    train
)

cuda:0
Building Model Backbone for ../input/shopee-models/paraphrase-xlm-r-multilingual-v1 model


get_bert_embeddings: 100%|████████████████████| 209/209 [00:04<00:00, 48.93it/s]


Finding similar by cosine KNN..., len of train: 6644, KNN=55
CV score for xlm_arcface_0.4 =  0.9644504530765503


In [4]:
import eca_nfnet_l0_arc_face

train = eca_nfnet_l0_arc_face.eval(
    train, destination="oof_nfnet_arcface", threshold=0.36
)

Building Model Backbone for eca_nfnet_l0 model


  0%|          | 0/831 [00:00<?, ?it/s]

Our image embeddings shape is (6644, 11014)
got embeddings! threshold=0.36
image embeddings shape (6644, 11014)
Finding similar by cosine KNN..., len of train: 6644, KNN=55
CV score for eca_nfnet_l0_arcface_0.36 =  0.8700010965741974


In [7]:
import numpy as np
def combine_for_sub(row):
    x=row.oof_xlm_arcface.split(' ') + row.oof_nfnet_arcface.split(' ')
    # x = np.concatenate([row.oof_xlm_arcface, row.oof_nfnet_arcface])
    return ' '.join( set(x) )

In [9]:
train.head()
train['matches'] = train.apply(combine_for_sub,axis=1)
train.to_csv(DATA_PATH+"res.csv")

In [10]:
import eval_preds
train['precision'],train['recall'],train['f1'] =  eval_preds.get_score(train['target'], train["matches"])
print(train.f1.mean())